In [18]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

from scipy.stats import entropy


# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
# def change_weights(X_train,y_train,X_test,y_test,weights):

In [19]:
pk = np.array([1/5, 2/5, 2/5])  # fair coin
H = entropy(pk)
print(H)

1.0549201679861442


In [20]:

def calculate_probabilities(list_labels, uniq_labels):
    '''
    Author: Sara Nassar 
    this function calculates the probabilities of each label in the list of labels
    it is calculated by number of labels in class A/all labels
    number of labels in class B/all labels
    and so on
    '''
    
    # A dictionary to store the probabilities
    probabilities = dict.fromkeys(uniq_labels, 0)
    
    # Total number of labels
    total_labels = len(list_labels)
    
    for label in uniq_labels:
        # Counting the number of times the label occurs in the list
        count = list_labels.count(label)
        
        # Calculating the probability of the label
        probability = count / total_labels
        
        # Storing the calculated probability in the dictionary
        probabilities[label] = probability
        
    return probabilities    
    
    
# test your function
list_labels=[1,2,0,1,2,0]
uniq_labels=[0,1,2]
print(calculate_probabilities(list_labels,uniq_labels))
# this should print somehting like 0.33,0.33,0.33


{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}


In [21]:

def calc_entropy_from_probabilities(list_probas):
    '''
    Author: Sara Nassar 
    list_probas is the list of probabiities
    the formula for entropy is
    sum(-proba*log(proba))
    
    '''
    
    entropy_value = 0

    for proba in list_probas:
        # If the probability is not zero
        if proba != 0:
            entropy_value += -proba * np.log(proba)
     
    return entropy_value


# test your function
list_probas=[1/5, 2/5, 2/5]
print(calc_entropy_from_probabilities(list_probas))
# above should print 1.054...

1.0549201679861442


In [22]:
def information_gain(old_entropy,new_entropies,count_items):
    '''
    Author: Sara Nassar 
    from the list of new entropies, calculate the overall new entropy
    
    formula is something like:
    overall_new_entropy = entropy1*proportion1 + entropy2*proportion2+ entropy3*proportion3 ...
    
    igain=old_entropy-overall_new_entropy
    '''
    
    overall_new_entropy = 0
    
    # Calculating the total number of items
    total_items = sum(count_items)
    
    for i in range(len(new_entropies)):
        # Calculating the proportion of items in the current partition
        proportion = count_items[i] / total_items
        
        # Adding the entropy of the current partition weighted by its proportion to the overall new entropy
        overall_new_entropy += new_entropies[i] * proportion
        
    # Calculating the information gain
    information_gain = old_entropy - overall_new_entropy
    
    return information_gain

#test your function
old_entropy=1
new_entropies=[0,0.65]
count_items=[4,6]
print(information_gain(old_entropy,new_entropies,count_items))
# above should print 0.61
    
    
    

0.61


In [23]:

def initialize_weights(number_features):
    '''
    the first set of weights corresponding to the features
    For now, it defaults to 2
    '''
    
    weights=np.array([np.random.uniform() for i in range(number_features)])
    return weights
    

In [24]:
num_feats=X_train.shape[1]
print(initialize_weights(num_feats))

[0.55252942 0.1611271  0.56854896 0.48570987]


In [25]:
def get_entropy_from_groups(new_entropies,count_items):
    overall_new_entropy = 0
    
    # Calculating the total number of items
    total_items = sum(count_items)
    
    for i in range(len(new_entropies)):
        # Calculating the proportion of items in the current partition
        proportion = count_items[i] / total_items
        
        # Adding the entropy of the current partition weighted by its proportion to the overall new entropy
        overall_new_entropy += new_entropies[i] * proportion
        
    return overall_new_entropy    

def get_entropy(threshold,res,y_test):

    # make two groups
    group1=[]
    group2=[]

    for i in range(res.shape[0]):
        if res[i]<threshold:
            group1.append(y_test[i])
        else:
            group2.append(y_test[i])




    proba_gr1=calculate_probabilities(group1,np.unique(group1).tolist())
    proba_gr1=list(proba_gr1.values()) 
    entropy_group1=calc_entropy_from_probabilities(proba_gr1)
    count_group1=len(proba_gr1)

    proba_gr2=calculate_probabilities(group2,np.unique(group2).tolist())
    proba_gr2=list(proba_gr2.values()) 
    entropy_group2=calc_entropy_from_probabilities(proba_gr2)
    count_group2=len(proba_gr2)

    new_entropies=[entropy_group1,entropy_group2]
    count_items=[count_group1,count_group2]
    overall_new_entropy=get_entropy_from_groups(new_entropies,count_items)
    return overall_new_entropy


### Task4: PSO Implementation

#### Modified the entropy function to get a vector of entropies for n particles

In [26]:
def objective_fn(param1,param2,X,y):
    '''
    param1 and param2 are the parameters that we want to optimize
    say param1 is the weight vector and  param2 is the threshold
    '''
    # Authon Bilal
    # multiply the weights with each feature and calculate the sum
    res=np.sum(X * param1, axis=1)  
#     print(res)
    #calculate entropy: hint: use the get_entropy function
    entropy= get_entropy(param2,res,y)
    # you only need to pass the threshold, res vector and the y
    return entropy
    
    
def objective_fn_vector(params1,params2,X,y):
    '''
    params1 is an array of weight vectors
    params2 is an array of thresholds
    '''
    results=[]
    for i in range(params1.shape[0]):
        param1=params1[i]
        param2=params2[i]
        # call the objective_fn above to get the entropy
        res=objective_fn(param1,param2,X,y)
#         print(param2,res)
        results.append(res)
    
    return np.array(results)
    

In [27]:
### Below we just randomly assign 100 particles and see if we can find the global minimum.
### THis is just to check

In [28]:
# Authon Bilal
params1=initialize_weights(400).reshape(100,4)
# call the initialize_weights function above

params2=np.random.uniform(low=0, high=1, size=100)
# use the np.random.uniform() function

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1=np.array(params1)
params2=np.array(params2)

print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 4)
Shape of params 2 (thresholds) (100,)


In [29]:
# Authon Bilal
z = objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum that is using the minimum if params1 and params2
param1_min =  params1[z.argmin()]# use z.argmin() to access best params1
param2_min = params2[z.argmin()]# use z.argmin() to access best params2

print("param1_min",param1_min,"param2_min",param2_min)


param1_min [0.40984647 0.20704059 0.95602267 0.40601118] param2_min 0.5911130758518441


In [30]:
### Setting up the particles and other parameters now

In [31]:
# Authon Bilal
# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1= initialize_weights(n_particles*4).reshape(n_particles,4)# a vector of shape n_particles,4
# call the initialize_weights function above

params2=np.random.uniform(low=0, high=1, size=n_particles)# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 4) params2 shape is  (20,)


In [32]:
# Authon Bilal
# define velocity of each weight of every particle
V_param1 =  initialize_weights(n_particles*4).reshape(n_particles,4) # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 =np.random.uniform(low=0, high=1, size=n_particles) # shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.67013703 0.82232957 0.64329013 0.70573338 0.73886477 0.82232957
 1.09729975 0.5237323  0.77244152 0.4620281  0.86703698 0.81919055
 1.09729975 0.81919055 0.73355763 0.74030523 0.82232957 0.73805779
 1.09729975 0.68309963]
gbest obj value among all 20 particles is as follows 0.4620281046196322


### the update function

In [33]:
def update():
    "Function to do one iteration of particle swarm optimization"
    # Authon Bilal
    global V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            # update the three lines below
            pbest[0][i]= params1[i]# # update pbest[0][i] with value of params1[i]
            pbest[1][i]= params2[i]# update pbest[1][i] 
            pbest_obj[i]= obj[i] # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj
 


In [34]:
for i in range(100):
    update()
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], objective_fn(param1_min, param2_min, X_train, y_train)))

PSO found best solution at f((array([ 0.77394123, -0.1722639 ,  0.88045306,  0.74404172]), 0.7465800267589203))=0.4161039895073432
Global optimal at f([array([0.40984647, 0.20704059, 0.95602267, 0.40601118]), 0.5911130758518441])=0.4620281046196322
